In [1]:
# load dependecies
#import cv2 # image procesing and computer vision
#import numpy as np # mathematical and algebra 
#import matplotlib.pyplot as plt # vizualization
#import pytesseract as pt    # OCR image to data (text, and others)
#from pytesseract import Output  # 
#import nltk     # natural language processing
#from nltk.tag.stanford import StanfordNERTagger # named entity recognition dependecy
#import os   # 


# Beta code # 
## Extracting  ##

In [2]:
## load depedencies 
import numpy as np # math library
import cv2 # visual computational 
import matplotlib.pyplot as plt # vizualization
import os # directory management
from configobj import ConfigObj # load rules
import lossrun

## Read files ##

In [4]:
## Read raw txt info in pdf report

PATH_txt = os.path.join('.','data','txt','')
PATH_image = os.path.join('.','data','images','')

# pdf raw file no ext 
# input file name


#__________________________________________________
pdf_file = '12 - 13 loss runs - Focus Nursing CHG770310301 11-1-12-13_YES0'
#___________________________________________________


#  Read text file 
txt_file = PATH_txt + pdf_file + '.txt'
#image file
image_file = PATH_image + pdf_file + '.jpg'

# Read as dict format txt file
txt_dict = lossrun.read_dict(txt_file)

# load configuration file prototype
template_rules = ConfigObj('config_file.ino')
